# Testing a Pre-Trained Network

Now that we know how to train an LSTM network, let's see try seeing the results on our own input text!

As we showed at the end of the Oriole notebook, we can load in a pretrained LSTM network using Tensorflow's Saver object. Before creating that object, we first have to create our Tensorflow graph. 

First, we declare some of our hyperparamters. 

In [251]:
numDimensions = 300
maxSeqLength = 25
batchSize = 24
lstmUnits = 64
numClasses = 2
iterations = 100000

Then, we'll load in our data structures.

In [252]:
import numpy as np
wordsList = np.load('wordsList.npy').tolist()
wordsList = [word.decode('UTF-8') for word in wordsList] #Encode words as UTF-8
wordVectors = np.load('wordVectors.npy')

Next, we'll create our graph. This is the same code as we saw in the main Oriole notebook. 

In [253]:
import tensorflow as tf
tf.reset_default_graph()

labels = tf.placeholder(tf.float32, [batchSize, numClasses])
input_data = tf.placeholder(tf.int32, [batchSize, maxSeqLength])

data = tf.Variable(tf.zeros([batchSize, maxSeqLength, numDimensions]),dtype=tf.float32)
data = tf.nn.embedding_lookup(wordVectors,input_data)

lstmCell = tf.contrib.rnn.BasicLSTMCell(lstmUnits)
lstmCell = tf.contrib.rnn.DropoutWrapper(cell=lstmCell, output_keep_prob=0.25)
value, _ = tf.nn.dynamic_rnn(lstmCell, data, dtype=tf.float32)

weight = tf.Variable(tf.truncated_normal([lstmUnits, numClasses]))
bias = tf.Variable(tf.constant(0.1, shape=[numClasses]))
value = tf.transpose(value, [1, 0, 2])
last = tf.gather(value, int(value.get_shape()[0]) - 1)
prediction = (tf.matmul(last, weight) + bias)

correctPred = tf.equal(tf.argmax(prediction,1), tf.argmax(labels,1))
accuracy = tf.reduce_mean(tf.cast(correctPred, tf.float32))

Now, we load in the network. 

In [254]:
sess = tf.InteractiveSession()
saver = tf.train.Saver()
saver.restore(sess, tf.train.latest_checkpoint('models'))

INFO:tensorflow:Restoring parameters from models/pretrained_lstm.ckpt-90000


Before we input our own text, let's first define a couple of functions. The first is a function to make sure the sentence is in the proper format, and the second is a function that obtains the word vectors for each of the words in a given sentence. 

In [255]:
# Removes punctuation, parentheses, question marks, etc., and leaves only alphanumeric characters
import re
strip_special_chars = re.compile("[^A-Za-z0-9 ]+")

def cleanSentences(string):
    string = string.lower().replace("<br />", " ")
    return re.sub(strip_special_chars, "", string.lower())

def getSentenceMatrix(sentence):
    arr = np.zeros([batchSize, maxSeqLength])
    sentenceMatrix = np.zeros([batchSize,maxSeqLength], dtype='int32')
    cleanedSentence = cleanSentences(sentence)
    split = cleanedSentence.split()
    for indexCounter, word in enumerate(split):
        if indexCounter >= maxSeqLength:
            break
        try:
            sentenceMatrix[0,indexCounter] = wordsList.index(word)
        except ValueError:
            sentenceMatrix[0,indexCounter] = 399999 #Vector for unkown words
    return sentenceMatrix

Now, we can create our input text. Try modifying the value for inputText, and see how the outputs change!

In [256]:
inputText = "That movie was terrible."
inputMatrix = getSentenceMatrix(inputText)

In [257]:
predictedSentiment = sess.run(prediction, {input_data: inputMatrix})[0]
# predictedSentiment[0] represents output score for positive sentiment
# predictedSentiment[1] represents output score for negative sentiment

if (predictedSentiment[0] > predictedSentiment[1]):
    print("Positive Sentiment")
else:
    print("Negative Sentiment")

Negative Sentiment


In [258]:
secondInputText = "That movie was the best one I have ever seen."
secondInputMatrix = getSentenceMatrix(secondInputText)

In [259]:
predictedSentiment = sess.run(prediction, {input_data: secondInputMatrix})[0]
if (predictedSentiment[0] > predictedSentiment[1]):
    print("Positive Sentiment")
else:
    print("Negative Sentiment")

Positive Sentiment


## Prediction

In [260]:
import pandas as pd

In [261]:
twitter_test_data_path = "twitter_data/testing.csv"

In [262]:
tw_test_df = pd.read_csv(twitter_test_data_path, engine='python', header=None)[:10000]

In [263]:
print(len(tw_test_df))
tw_test_df.head()

10000


,0,1,2,3,4,5
0,4,1834009132,Mon May 18 01:16:44 PDT 2009,NO_QUERY,Pete_Hyena,uguigkuygffjhfhjfhj is about how i feel today....
1,0,2058782290,Sat Jun 06 15:37:01 PDT 2009,NO_QUERY,serioussarcasm,My voice is so tired from singing..i'm gonna k...
2,0,2261930956,Sat Jun 20 21:35:32 PDT 2009,NO_QUERY,nowthatsasandra,sad part im not going to cry.... tears*
3,0,2061819960,Sat Jun 06 21:41:19 PDT 2009,NO_QUERY,beautifulcity,WHY ARE DRUNKS SO LOUD. stupid dock right next...
4,0,1678243705,Sat May 02 06:02:51 PDT 2009,NO_QUERY,geminilady75,@MariaBeckett Sorry to hear that hun


In [264]:
expected_labels = list(tw_test_df[0])
len(expected_labels)

10000

In [265]:
#(0 = negative, 4 = positive)
predicted_labels = []
for index, row in tw_test_df.iterrows():
    if index % 1000 == 0:
        print("index:", index)
    text = row[5]
    input_matrix = getSentenceMatrix(text)
    predictedSentiment = sess.run(prediction, {input_data: input_matrix})[0]
    if (predictedSentiment[0] > predictedSentiment[1]):
        #pos
        predicted_labels.append(4)
        #print("Positive Sentiment")
    else:
        #neg
        predicted_labels.append(0)
        #print("Negative Sentiment")

index: 0
index: 1000
index: 2000
index: 3000
index: 4000
index: 5000
index: 6000
index: 7000
index: 8000
index: 9000


## Metrics

In [266]:
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix

In [267]:
target_names = ['negative', 'positive']
print(classification_report(expected_labels, predicted_labels, target_names=target_names))

             precision    recall  f1-score   support

   negative       0.79      0.81      0.80      4985
   positive       0.80      0.78      0.79      5015

avg / total       0.80      0.80      0.80     10000



In [268]:
confusion_matrix(expected_labels, predicted_labels)

array([[4024,  961],
       [1082, 3933]])

## Save Result

In [269]:
tw_test_df[6] = predicted_labels

In [270]:
tw_test_df.tail()[[0, 5, 6]]

,0,5,6
9995,0,"@richardlai no, but I believe a lot of iPhone ...",0
9996,0,sorry that grandma bell died,0
9997,0,Going to see what the big fuss is about. So it...,4
9998,4,"@chloes2davis oh good, saves a lot of effort",4
9999,0,"@skubastevee Aww, wish my dog had playmates......",0


In [271]:
tw_test_df.to_csv("tw_test_predicted.csv", index=None, header=None)